<a href="https://colab.research.google.com/github/mikancode/daily-coding/blob/main/20251230-corp-url-finder/01_search_official_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
from google.colab import userdata

In [ ]:
import json

target_campany_name = "日本郵便"

url = "https://trial.api.info.gbiz.go.jp/hojin/v2/hojin"

params = {
    "page":1,
    "limit":1000,
    "metadata_flg":"false",
    "name":target_campany_name
}

headers = {"X-hojinInfo-api-token": userdata.get('GBIZ_API_KEY')}


try:
    response = requests.get(url, params=params, headers=headers, timeout=10)  # タイムアウト設定
    response.raise_for_status()  # ステータスコードが200番台以外の場合に例外を発生
    data = response.json()  # レスポンスボディをJSONとして解析
    # dataを使った処理...
    display(data)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

In [ ]:
hji = data['hojin-infos']
print(hji)
full_url = f"{url}/{hji[0]['corporate_number']}"

print(full_url)

params = {
    "metadata_flg":"true"
}

headers = {"X-hojinInfo-api-token": userdata.get('GBIZ_API_KEY')}


try:
    response = requests.get(full_url, params=params, headers=headers, timeout=10)  # タイムアウト設定
    response.raise_for_status()  # ステータスコードが200番台以外の場合に例外を発生
    data = response.json()  # レスポンスボディをJSONとして解析
    # dataを使った処理...
    display(data)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

In [ ]:
import pandas as pd

# 入力データ（data変数に格納されている前提）
hojin_list = data['hojin-infos']

rows = []
for hojin in hojin_list:
    corporate_number = hojin.get('corporate_number')
    meta = hojin.get('meta-data', {})

    # メインの各キーをループ
    for key, value in hojin.items():
        # meta-data自体や、リスト/辞書形式の項目は除外（必要に応じて調整）
        if key in ['meta-data', 'industry', 'business_items']:
            continue

        rows.append({
            'corporate_number': corporate_number,
            'item_name': key,
            'value': value,
            'data_quality': meta.get('data_quality', {}).get(key),
            'source': meta.get('source', {}).get(key),
            'import_frequency': meta.get('import_frequency', {}).get(key),
            'last_update': meta.get('last_update_date', {}).get(key)
        })

df_result = pd.DataFrame(rows)

# 表示
display(df_result)

In [ ]:
from urllib.robotparser import RobotFileParser

def can_fetch(url, user_agent='*'):
    # robots.txtのURLを作成
    from urllib.parse import urlparse
    parsed_url = urlparse(url)
    robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"

    rp = RobotFileParser()
    rp.set_url(robots_url)
    try:
        rp.read()
    except:
        # robots.txtが読めない場合は保守的にFalseにするか、Allowとみなすか判断
        return False

    # 指定したURLが許可されているか確認
    return rp.can_fetch(user_agent, url)

# 使用例
target_url = df_result[df_result["item_name"] == "company_url"].iloc[0]["value"]
if can_fetch(target_url):
    print("スクレイピング可能です")
else:
    print("このボットでのアクセスは拒否されています")

In [ ]:
for hojin in hojin_list:
    company_url = hojin.get('company_url')

try:
    response = requests.get(company_url, timeout=10)  # タイムアウト設定
    response.raise_for_status()  # ステータスコードが200番台以外の場合に例外を発生
    response.encoding = 'utf-8'
    html_content = response.text  # レスポンスをHTMLとして解析
    # dataを使った処理...
    display(html_content)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


In [ ]:
from bs4 import BeautifulSoup

# スクレイピング対象の URL にリクエストを送り HTML を取得する

# レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(html_content, 'html.parser')

# title タグの文字列を取得する
title_text = soup.find('title').get_text()
print(title_text)

# ページに含まれるリンクを全て取得する
links = [url.get('href') for url in soup.find_all('a')]
display(links)
